In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [0]:
football = pd.read_csv("/content/FIFA 2018 Statistics.csv")

In [0]:
numerical_features   = football.select_dtypes(include = [np.number]).columns
categorical_features = football.select_dtypes(include= [np.object]).columns

In [0]:
football['Man of the Match'] = football['Man of the Match'].map({'Yes': 1, 'No': 0})
football["PSO"] = football["PSO"].map({'Yes': 1, 'No': 0})

In [0]:
corr = football.corr()
corr = corr.filter(items = ['Man of the Match'])
#plt.figure(figsize=(15,8))
#sns.heatmap(corr, annot=True)
print(corr)


                        Man of the Match
Goal Scored                     0.522193
Ball Possession %               0.110393
Attempts                        0.176425
On-Target                       0.291346
Off-Target                      0.152979
Blocked                        -0.019582
Corners                         0.173148
Offsides                        0.026288
Free Kicks                      0.119534
Saves                          -0.034443
Pass Accuracy %                 0.109709
Passes                          0.101421
Distance Covered (Kms)          0.015353
Fouls Committed                -0.129047
Yellow Card                    -0.147934
Yellow & Red                    0.000000
Red                            -0.125988
Man of the Match                1.000000
1st Goal                        0.009802
PSO                             0.000000
Goals in PSO                    0.038873
Own goals                            NaN
Own goal Time                  -0.186824


In [0]:
for i in corr.index:
  if (((corr.loc[i].at["Man of the Match"] < 0.12) and (corr.loc[i].at["Man of the Match"] > 0)) or ((corr.loc[i].at["Man of the Match"] > -0.12) and (corr.loc[i].at["Man of the Match"] <= 0))):
    corr.drop(index = i, inplace = True)
corr.drop(index = ["Own goals", "Own goal Time"], inplace = True)
print(corr)

                  Man of the Match
Goal Scored               0.522193
Attempts                  0.176425
On-Target                 0.291346
Off-Target                0.152979
Corners                   0.173148
Fouls Committed          -0.129047
Yellow Card              -0.147934
Red                      -0.125988
Man of the Match          1.000000


In [0]:
for i in football.columns:
  if ((i not in corr.index) and (i not in ["Team", "Opponent"])):
    print(i)
    football.drop(columns = i, inplace = True)
print(football.head())

Date
Ball Possession %
Blocked
Offsides
Free Kicks
Saves
Pass Accuracy %
Passes
Distance Covered (Kms)
Yellow & Red
1st Goal
Round
PSO
Goals in PSO
Own goals
Own goal Time
           Team      Opponent  Goal Scored  ...  Yellow Card  Red  Man of the Match
0        Russia  Saudi Arabia            5  ...            0    0                 1
1  Saudi Arabia        Russia            0  ...            0    0                 0
2         Egypt       Uruguay            0  ...            2    0                 0
3       Uruguay         Egypt            1  ...            0    0                 1
4       Morocco          Iran            0  ...            1    0                 0

[5 rows x 11 columns]


In [0]:
football = pd.get_dummies(football)
print(football.head())

   Goal Scored  Attempts  ...  Opponent_Tunisia  Opponent_Uruguay
0            5        13  ...                 0                 0
1            0         6  ...                 0                 0
2            0         8  ...                 0                 1
3            1        14  ...                 0                 0
4            0        13  ...                 0                 0

[5 rows x 73 columns]


In [0]:
Y = football.loc[:, ["Man of the Match"]]
X = football.drop(columns = "Man of the Match", axis = 1)
print(X.shape, Y.shape)

(128, 72) (128, 1)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 42)
print (X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(96, 72) (32, 72) (96, 1) (32, 1)


In [0]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, max_iter = 1000).fit(X_train, Y_train)
Y_pred = clf.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [0]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, Y_pred)
print(cm)
#LogisticRegression

[[11  3]
 [ 5 13]]


In [0]:
from sklearn import svm
clf = svm.SVC(random_state=0, kernel="linear").fit(X_train, Y_train)
Y_pred = clf.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
cm = confusion_matrix(Y_test, Y_pred)
print(cm)
#SVM

[[12  2]
 [ 5 13]]


In [0]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion="entropy", max_features=7, random_state=0).fit(X_train, Y_train)
Y_pred = clf.predict(X_test)

In [0]:
cm = confusion_matrix(Y_test, Y_pred)
print(cm)
#Decision Tree Classifier 1

[[12  2]
 [ 4 14]]


In [0]:
clf = tree.DecisionTreeClassifier(criterion="entropy", max_features=67, splitter="random", random_state=0).fit(X_train, Y_train)
Y_pred = clf.predict(X_test)

In [0]:
cm = confusion_matrix(Y_test, Y_pred)
print(cm)
#Decision Tree Classifier 2 best config - criterion="entropy", max_features=67, splitter="random", random_state=0

[[12  2]
 [ 3 15]]


In [0]:
#for i in range(1, 73):
#  clf = tree.DecisionTreeClassifier(criterion="entropy", splitter="random", max_features=i, random_state=0).fit(X_train, Y_train)
#  Y_pred = clf.predict(X_test)
#  cm = confusion_matrix(Y_test, Y_pred)
#  accuracy = (cm[0][0] + cm[1][1])/(cm[1][0] + cm[0][0] + cm[1][1] + cm[0][1])
#  print(accuracy, i)

0.625 1
0.5625 2
0.53125 3
0.4375 4
0.75 5
0.8125 6
0.5625 7
0.71875 8
0.625 9
0.71875 10
0.625 11
0.625 12
0.75 13
0.71875 14
0.625 15
0.65625 16
0.71875 17
0.65625 18
0.53125 19
0.78125 20
0.625 21
0.75 22
0.6875 23
0.78125 24
0.8125 25
0.75 26
0.65625 27
0.75 28
0.78125 29
0.6875 30
0.75 31
0.65625 32
0.6875 33
0.71875 34
0.65625 35
0.75 36
0.6875 37
0.78125 38
0.78125 39
0.8125 40
0.65625 41
0.75 42
0.71875 43
0.71875 44
0.71875 45
0.6875 46
0.65625 47
0.75 48
0.5 49
0.71875 50
0.625 51
0.75 52
0.78125 53
0.71875 54
0.84375 55
0.6875 56
0.8125 57
0.6875 58
0.65625 59
0.65625 60
0.625 61
0.6875 62
0.59375 63
0.5625 64
0.6875 65
0.75 66
0.84375 67
0.71875 68
0.71875 69
0.8125 70
0.8125 71
0.71875 72


In [0]:
from sklearn import ensemble
from sklearn.metrics import confusion_matrix
clf = ensemble.RandomForestClassifier(n_estimators=1000).fit(X_train, Y_train.to_numpy().ravel())
Y_pred = clf.predict(X_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.78125


In [0]:
cm = confusion_matrix(Y_test, Y_pred)
print(cm)
#Random Forest Classifier 1

[[12  2]
 [ 6 12]]


In [0]:
clf = ensemble.RandomForestClassifier(n_estimators=1000, criterion = "entropy", max_features = "log2", random_state = 0).fit(X_train, Y_train.to_numpy().ravel())
Y_pred = clf.predict(X_test)
cm = confusion_matrix(Y_test, Y_pred)
print(cm)
#Random Forest Classifier 2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


[[12  2]
 [ 5 13]]


In [0]:
clf = ensemble.RandomForestClassifier(n_estimators=1000, criterion = "gini", max_features = 14, random_state = 0).fit(X_train, Y_train.to_numpy().ravel())
Y_pred = clf.predict(X_test)
cm = confusion_matrix(Y_test, Y_pred)
accuracy = (cm[0][0] + cm[1][1])/(cm[1][0] + cm[0][0] + cm[1][1] + cm[0][1])
print (cm)
#Random Forest Classifier 3

0.8125 1305
[[12  2]
 [ 4 14]]


In [0]:
from sklearn import naive_bayes
clf = naive_bayes.GaussianNB().fit(X_train, Y_train.to_numpy().ravel())
Y_pred = clf.predict(X_test)
cm = confusion_matrix(Y_test, Y_pred)
print(cm)
#Gaussian NB

[[ 4 10]
 [ 4 14]]


In [0]:
clf = naive_bayes.MultinomialNB().fit(X_train, Y_train.to_numpy().ravel())
Y_pred = clf.predict(X_test)
cm = confusion_matrix(Y_test, Y_pred)
print(cm)
#Multinomial NB

[[ 9  5]
 [ 5 13]]


In [0]:
clf = naive_bayes.ComplementNB().fit(X_train, Y_train.to_numpy().ravel())
Y_pred = clf.predict(X_test)
cm = confusion_matrix(Y_test, Y_pred)
print(cm)
#Complement NB

[[ 9  5]
 [ 5 13]]


In [0]:
clf = naive_bayes.BernoulliNB().fit(X_train, Y_train.to_numpy().ravel())
Y_pred = clf.predict(X_test)
cm = confusion_matrix(Y_test, Y_pred)
print(cm)
#Bernoulli NB

[[11  3]
 [ 5 13]]


In [0]:
from sklearn import ensemble
from sklearn.metrics import confusion_matrix
for i in range(100, 10000, 100):
  for j in range(1, 73):
    clf = ensemble.RandomForestClassifier(n_estimators=i, criterion = "entropy", max_features = j, random_state = 0).fit(X_train, Y_train.to_numpy().ravel())
    Y_pred = clf.predict(X_test)
    cm = confusion_matrix(Y_test, Y_pred)
    accuracy = (cm[0][0] + cm[1][1])/(cm[1][0] + cm[0][0] + cm[1][1] + cm[0][1])
    if (accuracy >= 0.84375):
      print(accuracy, i, j, "entropy")


    clf = ensemble.RandomForestClassifier(n_estimators=i, criterion = "gini", max_features = j, random_state = 0).fit(X_train, Y_train.to_numpy().ravel())
    Y_pred = clf.predict(X_test)
    cm = confusion_matrix(Y_test, Y_pred)
    accuracy = (cm[0][0] + cm[1][1])/(cm[1][0] + cm[0][0] + cm[1][1] + cm[0][1])
    if (accuracy >= 0.84375):
      print(accuracy, i, j, "gini")

#best config: random forest - n_estimators=100, criterion = "gini", max_features = 38, random_state = 0

0.84375 100 38 gini
0.84375 100 41 gini
0.84375 100 61 gini


KeyboardInterrupt: ignored